# DNNで二値分類をする
keras_tunerを使用して、ハイパーパラメータのチューニングを行う。

### 大まかな処理の流れ
1. データの取り込み
2. データの整形（トレーニングデータとテストデータを作成）
3. モデルの定義
4. チューナーをインスタンス化して、ハイパーパラメータのチューニングする
5. チューニングしたハイパーパラメータでモデルを作成し、トレーニングする
6. 作成したモデルを保存する
7. 保存したモデルをお読み込み、予測したいデータで予測する

## 使用ライブラリのインポート

In [1]:
import tensorflow as tf
import pandas as pd
from keras.utils import FeatureSpace
import keras_tuner
from tensorflow import keras
import json

## 1. データの取り込み

In [2]:
# 予測に使用したいデータに応じて変更する
with open('index_data.json') as f:
    json_data = json.load(f)

dataframe = pd.read_json(json.dumps(json_data['data']))

## 2. データの確認
省略可能

In [3]:
print(dataframe.shape)

(1000, 21)


In [4]:
dataframe.head()

,bankLink,waitTime,returnRate,ppu,totalImgByte,imageNum,newVisitRate,mainContentHeight,imgLoadTime,jsResponseTime,...,cssByte,cssNum,jsByte,jsNum,keywordNum,headLineNum,isHeadKLine1Keyword,titleNum,isTitleKeyword,isSiteIndex
0,1389,132,27,6,4727259,41,77,4160,7.268064,346,...,62300,3,947599,29,19,14,1,163,0,0
1,270,299,30,2,5716184,191,47,8643,8.864082,782,...,118108,9,2584518,91,3,35,1,179,0,0
2,77,58,34,23,4739805,180,54,8369,6.010119,657,...,136080,3,1236224,41,14,41,0,83,0,0
3,1765,262,22,28,681397,22,33,5794,9.432883,758,...,105061,11,1132534,75,20,43,0,188,0,1
4,628,114,45,30,671171,15,32,6095,6.012446,224,...,52964,18,3709963,68,20,13,0,199,1,1


## 3. トレーニングデータと検証データの作成

In [5]:
# データフレームから検証用としてランダムに20％取り出す
# random_stateを固定しているので、実行ごとに検証データとして取り出すデータは同じ
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

Using 800 samples for training and 200 for validation


In [6]:
# 予測に使用したいデータに応じて変更する
predict_value = "isSiteIndex"

# データフレームからデータセットに変換
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop(predict_value)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [7]:
# データセットの中身の確認
for x, y in train_ds.take(1):
    print("Input:", x)
    print(predict_value, y)

Input: {'bankLink': <tf.Tensor: shape=(), dtype=int64, numpy=1424>, 'waitTime': <tf.Tensor: shape=(), dtype=int64, numpy=102>, 'returnRate': <tf.Tensor: shape=(), dtype=int64, numpy=86>, 'ppu': <tf.Tensor: shape=(), dtype=int64, numpy=28>, 'totalImgByte': <tf.Tensor: shape=(), dtype=int64, numpy=5008574>, 'imageNum': <tf.Tensor: shape=(), dtype=int64, numpy=87>, 'newVisitRate': <tf.Tensor: shape=(), dtype=int64, numpy=31>, 'mainContentHeight': <tf.Tensor: shape=(), dtype=int64, numpy=5157>, 'imgLoadTime': <tf.Tensor: shape=(), dtype=float64, numpy=9.091684422266589>, 'jsResponseTime': <tf.Tensor: shape=(), dtype=int64, numpy=987>, 'htmlTotalByte': <tf.Tensor: shape=(), dtype=int64, numpy=41589>, 'cssByte': <tf.Tensor: shape=(), dtype=int64, numpy=143579>, 'cssNum': <tf.Tensor: shape=(), dtype=int64, numpy=16>, 'jsByte': <tf.Tensor: shape=(), dtype=int64, numpy=610519>, 'jsNum': <tf.Tensor: shape=(), dtype=int64, numpy=66>, 'keywordNum': <tf.Tensor: shape=(), dtype=int64, numpy=9>, 'hea

In [8]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [9]:
# 予測に使用したいデータに応じて変更する
# 構造化データの前処理とエンコードを行う
# 各データの特徴を以下の分類に当てはめる。
# "integer_categorical"：数値（0,1など）カテゴリー、"string_categorical"：文字列カテゴリー、"float_discretized"：離散化する数値的特徴、"float_normalized"：正規化する数値的特徴
feature_space = FeatureSpace(
    features={
        # 数値カテゴリー特徴
        "isHeadKLine1Keyword": "integer_categorical",
        "isTitleKeyword": "integer_categorical",
        # 正規化する数値的特徴
        "bankLink": "float_normalized",
        "waitTime": "float_normalized",
        "returnRate": "float_normalized",
        "ppu": "float_normalized",
        "totalImgByte": "float_normalized",
        "imageNum": "float_normalized",
        "newVisitRate": "float_normalized",
        "mainContentHeight": "float_normalized",
        "imgLoadTime": "float_normalized",
        "jsResponseTime": "float_normalized",
        "htmlTotalByte": "float_normalized",
        "cssByte": "float_normalized",
        "cssNum": "float_normalized",
        "jsByte": "float_normalized",
        "jsNum": "float_normalized",
        "keywordNum": "float_normalized",
        "headLineNum": "float_normalized",
        "titleNum": "float_normalized",
    },
    # "concat"または"dict"。
    # "concat"では、すべての特徴が 1 つのベクトルに連結される。
    # "dict"では、個別にエンコードされた特徴のdictを返す (入力キーと同じキーを使用)。
    output_mode="concat",
)

In [10]:
train_ds_with_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_with_no_labels)
feature_space.save("indexfeaturespace.keras")

In [11]:
# トレーニングデータセットから１つ取り出して確認
for x, _ in train_ds.take(1):
    preprocessed_x = feature_space(x)
    print("preprocessed_x.shape:", preprocessed_x.shape)
    print("preprocessed_x.dtype:", preprocessed_x.dtype)

preprocessed_x.shape: (32, 24)
preprocessed_x.dtype: <dtype: 'float32'>


In [12]:
# num_parallel_callsで処理を並列化する。
# tf.data.AUTOTUNEは並列度をランタイムで良い感じに決めてくれる。
preprocessed_train_ds = train_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
# prefetchはGPUが計算している間にBatchデータをCPU側で用意しておく機能
preprocessed_train_ds = preprocessed_train_ds.prefetch(tf.data.AUTOTUNE)

preprocessed_val_ds = val_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_val_ds = preprocessed_val_ds.prefetch(tf.data.AUTOTUNE)

print(preprocessed_train_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 24), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


## 4. モデルを定義

In [13]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

# 予測に使用したいデータに応じて変更する
# チューニングしたいハイパーパラメーターを変更したい場合は変更する
def build_model(hp):
    model = tf.keras.Sequential()

    # 入力層
    model.add(tf.keras.Input(tensor=encoded_features))

    # 隠れ層
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=10) # max_valueに最適化した最大隠れ層数を指定
    for i in range(hp_n_hidden_layers):
        hp_units = hp.Int("units_%d" % (i + 1), min_value=32, max_value=512, step=32)
        hp_activation = hp.Choice("activation_%d" % (i + 1), ["relu", "tanh"]) # 活性化関数
        model.add(tf.keras.layers.Dense(hp_units, activation=hp_activation))
        model.add(tf.keras.layers.Dropout(hp.Choice(name="dropout_%d" % (i + 1), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))) # ドロップアウト率

    # 出力層
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    # 最適化アルゴリズム、損失関数、評価関数を指定してコンパイル
    # 最適化アルゴリズムを最適化
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    model.compile(
        # 最適化アルゴリズム
        optimizer=tf.keras.optimizers.Adam(hp_learning_rate),
        # 損失関数
        # 二値分類→binary_crossentropy、多クラス単一ラベル分類→categorical_crossentropy
        # 多クラス多ラベル分類→binary_crossentropy、回帰問題（任意の値）→mse、回帰問題（０～１の値）→mse / binary_crossentropy
        loss="binary_crossentropy",
        # 評価関数
        metrics=["accuracy"],
    )

    return model

## 5. チューナーをインスタンス化してハイパーパラメータのチューニングを実行

In [14]:
# 予測に使用したいデータに応じて変更する
# チューナーのインスタンス化
# RandomSearch、Hyperband、BayesianOptimization、Sklearnがある
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy", # 最適化する目標の名前：accuracy, loss, val_accuracy, val_loss
    max_trials=50, # チューニングする試行数
    executions_per_trial=2, # 各トライアルに構築して適合させる必要があるモデルの数
    overwrite=True,
    directory="./",
    project_name="index_predict",
)

In [15]:
# 検証損失が特定の値に達した時に、トレーニングを早期に停止するためのコールバック関数
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# TensorBoardを表示する場合、実行する
# 初回実行時などのフォルダが存在しない場合、エラーで起動できない
# http://localhost:6006/ にアクセスすると見れる
%load_ext tensorboard

%tensorboard --logdir="./index_predict/tmp/tb_logs" --bind_all

In [16]:
cur_dir_path = %pwd
tuner.search(
    preprocessed_train_ds, # トレーニングデータセット
    epochs=50, # 学習の繰り返す数
    validation_data=preprocessed_val_ds, # 検証データセット
    callbacks=[stop_early, keras.callbacks.TensorBoard(f"{cur_dir_path}/index_predict/tmp/tb_logs")], # TensorBoard表示用のコールバック関数（使用しない場合は消す）
)
best_model = tuner.get_best_models()[0]

Trial 50 Complete [00h 00m 06s]
val_accuracy: 0.6150000095367432

Best val_accuracy So Far: 0.6150000095367432
Total elapsed time: 00h 08m 06s
INFO:tensorflow:Oracle triggered exit


## 6. モデルをトレーニング

In [17]:
# 最適なハイパーパラメータ
best_hp = tuner.get_best_hyperparameters()[0]
# 最適なハイパーパラメータでモデルの作成
model = tuner.hypermodel.build(best_hp)
# モデルのトレーニング
model.fit(preprocessed_train_ds, epochs=50, validation_data=preprocessed_val_ds)

Epoch 1/50
25/25 [==============================] - 1s 13ms/step - loss: 0.7237 - accuracy: 0.5088 - val_loss: 0.6942 - val_accuracy: 0.5150
Epoch 2/50
25/25 [==============================] - 0s 8ms/step - loss: 0.7119 - accuracy: 0.5288 - val_loss: 0.7011 - val_accuracy: 0.4850
Epoch 3/50
25/25 [==============================] - 0s 8ms/step - loss: 0.6937 - accuracy: 0.5375 - val_loss: 0.7302 - val_accuracy: 0.4550
Epoch 4/50
25/25 [==============================] - 0s 8ms/step - loss: 0.7009 - accuracy: 0.5425 - val_loss: 0.7018 - val_accuracy: 0.5350
Epoch 5/50
25/25 [==============================] - 0s 8ms/step - loss: 0.6883 - accuracy: 0.5487 - val_loss: 0.7157 - val_accuracy: 0.4700
Epoch 6/50
25/25 [==============================] - 0s 8ms/step - loss: 0.6733 - accuracy: 0.6000 - val_loss: 0.7219 - val_accuracy: 0.5000
Epoch 7/50
25/25 [==============================] - 0s 8ms/step - loss: 0.6901 - accuracy: 0.5412 - val_loss: 0.7050 - val_accuracy: 0.5100
Epoch 8/50
25/25 [=

## 7. モデルの保存と概要

In [18]:
# モデルの保存
model.save("index_model")
# モデルの概要
model.summary()

INFO:tensorflow:Assets written to: index_model\assets


INFO:tensorflow:Assets written to: index_model\assets


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 288)               7200      
                                                                 
 dropout_5 (Dropout)         (None, 288)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               36992     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 128)               16512     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 320)              

In [19]:
# チューニングの概要
tuner.results_summary()

Results summary
Results in ./index_predict
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 49 summary
Hyperparameters:
n_hidden_layers: 5
units_1: 288
activation_1: tanh
dropout_1: 0.6
learning_rate: 0.001
units_2: 128
activation_2: relu
dropout_2: 0.3
units_3: 128
activation_3: tanh
dropout_3: 0.2
units_4: 320
activation_4: tanh
dropout_4: 0.1
units_5: 192
activation_5: relu
dropout_5: 0.1
units_6: 96
activation_6: relu
dropout_6: 0.3
units_7: 160
activation_7: relu
dropout_7: 0.1
units_8: 256
activation_8: relu
dropout_8: 0.9
units_9: 256
activation_9: relu
dropout_9: 0.1
units_10: 288
activation_10: relu
dropout_10: 0.2
Score: 0.6150000095367432

Trial 24 summary
Hyperparameters:
n_hidden_layers: 5
units_1: 128
activation_1: tanh
dropout_1: 0.1
learning_rate: 0.001
units_2: 320
activation_2: relu
dropout_2: 0.1
units_3: 384
activation_3: relu
dropout_3: 0.9
units_4: 32
activation_4: relu
dropout_4: 0.2
units_5: 416
activation_5: tanh
dropout_5: 0.9
unit